In [ ]:
%load_ext autoreload

In [ ]:
import os
os.chdir("../")

In [ ]:
%autoreload 2
import math
import numpy as np
from tqdm.auto import trange
from IPython.display import clear_output

import torch
import torch.nn as nn

import matplotlib.pyplot as plt
%matplotlib inline


import sys
sys.path.append('./external/sg3')
sys.path.append('./external/torch_hmc')


from sb.nn.reward import ClsReward
from sb.data.datasets import ClsRewardDist, MixOfGaussians
from sb.utils import plot_annotated_images

In [ ]:
device = torch.device('cuda:1')

# rwd_dist = ClsRewardDist('cifar10-gan-z256', 'cifar10-cls', 50, [5], 'sum', device)
rwd_dist = ClsRewardDist('cifar10-stylegan', 'cifar10-cls', 50, [5], 'sum', device)

In [ ]:
def compute_grad(fn, x):
    x_ = x.detach().clone().requires_grad_(True)    
    o = fn(x_)
    o.sum().backward()
    return o, x_.grad


def mala_correction(x, logp_x, grad_logp_x, y, logp_y, grad_logp_y, dt):
    adj = logp_y - logp_x + \
          (y - x - dt * grad_logp_x).pow(2).sum(1) / (4 * dt) - \
          (x - y - dt * grad_logp_y).pow(2).sum(1) / (4 * dt)

    adj = torch.minimum(torch.ones_like(adj), adj.exp())
    return torch.rand_like(adj) < adj

In [ ]:
method = 'ula' # ula, mala

# x = torch.randn(512, 256, device=device)
x = torch.randn(128, 512, device=device)
dt = 0.1
num_steps = 500
anneal_value = 0.1

lmbda = 0.9
grad_lmbda = 0.0
alpha = math.exp(math.log(anneal_value) / num_steps)

prev_z = None
ema_grad = 0
precision, mean_log_reward, adj_fracs = [], [], []
for it in trange(num_steps):
    if method == 'ula':
        _, grad = compute_grad(rwd_dist.log_density, x)
        ema_grad = grad_lmbda * ema_grad + (1 - grad_lmbda) * grad

        curr_z = torch.randn_like(x)
        z = curr_z if prev_z is None else (curr_z + prev_z) / 2
        prev_z = curr_z
        
        y = x + ema_grad * dt + math.sqrt(2 * dt) * z
        
        x = lmbda * x + (1 - lmbda) * y
        adj_fracs.append(1.0)
    
    elif method == 'mala':
        logp_x, grad_logp_x = compute_grad(rwd_dist.log_density, x)

        z = torch.randn_like(x)
        y = x + grad_logp_x * dt + math.sqrt(2 * dt) * z

        logp_y, grad_logp_y = compute_grad(rwd_dist.log_density, y)

        adj = mala_correction(x, logp_x, grad_logp_x, 
                            y, logp_y, grad_logp_y, 
                            dt)
        adj_fracs.append(adj.float().mean().item())
        x[adj] = y[adj]

    dt *= alpha 

    with torch.no_grad():
        img = rwd_dist.reward.generator(x)
        img = img * 0.5 + 0.5
        
        probas = rwd_dist.reward.classifier(img).softmax(dim=1)
        (p, c) = probas.max(dim=1)
        
        mean_log_rwd = probas[:, 5].log().mean().item()
        prc = (c == 5).float().mean().item()
    
    precision.append(prc)
    mean_log_reward.append(mean_log_rwd)

In [ ]:
f, ax = plt.subplots(3, 1, figsize=(12, 10))
names = ['Precision', 'Mean Log Reward', 'Acceptance Rate']

for i, graph  in enumerate([
    precision, mean_log_reward, adj_fracs
]):
    ax[i].set_title(names[i])
    ax[i].plot(graph)
plt.show()

In [ ]:
with torch.no_grad():
    img = rwd_dist.reward.generator(x)
    img = torch.clip(img * 0.5 + 0.5, 0, 1)
    probas = rwd_dist.reward.classifier(img).softmax(dim=1)
    (p, c) = probas.max(dim=1)
    
    mean_log_rwd = probas[:, 5].log().mean()
    precision = (c == 5).float().mean()


print(f"Precision: {precision:.2f}, Mean log reward: {mean_log_rwd:.3f}")
_ = plot_annotated_images(img[:16], (p[:16], c[:16]), n_col=4, figsize=(12, 12))

In [ ]:
import hamiltorch
hamiltorch.set_random_seed(123)

In [ ]:
x0 = torch.randn(512, device=device)

sampler=hamiltorch.Sampler.HMC
integrator=hamiltorch.Integrator.IMPLICIT


def _log_gensity(x):
    x = x.reshape(1, -1)
    density = rwd_dist.log_density(x)
    return density[0]

xt = params_hmc = hamiltorch.sample(
    log_prob_func=_log_gensity, 
    params_init=x0,  
    num_samples=64, 
    step_size=0.08, 
    num_steps_per_sample=10,
    sampler=sampler, 
    integrator=integrator,
)


In [ ]:
with torch.no_grad():
    x = torch.stack(xt[::4])
    img = rwd_dist.reward.generator(x)
    img = torch.clip(img * 0.5 + 0.5, 0, 1)
    probas = rwd_dist.reward.classifier(img).softmax(dim=1)

(p, c) = probas.max(dim=1)
mean_log_rwd = probas[:, 5].log().mean()

print(
    f"{img.shape[0]=} |",
    f" Precision: {(c == 5).float().mean():.2f} |",
    f"Mean log reward: {mean_log_rwd:.3f}"
)
_ = plot_annotated_images(img[:32], (p[:32], c[:32]), n_col=4, figsize=(12, 24))